In [1]:
! pip install fastbook

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 0.82.0 requires click<8.0,>=7.0, but you have click 8.1.3 which is incompatible.
You should consider upgrading via the 'c:\users\anjali goel\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.60.0
    Uninstalling tqdm-4.60.0:
      Successfully uninstalled tqdm-4.60.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.3
    Uninstalling dill-0.3.3:
      Successfully uninstalled dill-0.3.3
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 4.0.0
    Uninstalling pyarrow-4.0.0:
      Successfully uninstalled pyarrow-4.0.0


In [2]:
import fastbook

fastbook.setup_book()

In [3]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap = 'Greys')

In [4]:
path = untar_data(URLs.MNIST_SAMPLE)

In [5]:
Path.BASE_PATH = path

In [6]:
path.ls()

(#3) [Path('labels.csv'),Path('train'),Path('valid')]

In [7]:
(path/'train').ls()

(#2) [Path('train/3'),Path('train/7')]

In [9]:
threes = (path/'train/3').ls().sorted()
sevens = (path/'train/7').ls().sorted()

In [11]:
three_tensors = [tensor(Image.open(o)) for o in threes]
seven_tensors = [tensor(Image.open(o)) for o in sevens]

len(three_tensors), len(seven_tensors)

(6131, 6265)

In [13]:
stacked_threes = torch.stack(three_tensors) / 255
stacked_sevens = torch.stack(seven_tensors) / 255

In [17]:
valid_3 = torch.stack([tensor(Image.open(o)) for o in (path/'valid/3').ls()]) / 255
# valid_3 = valid_3.float() / 255
valid_7 = torch.stack([tensor(Image.open(o)) for o in (path/'valid/7').ls()]) / 255
# valid_7 = valid_7.float() / 255

valid_3.shape, valid_7.shape

(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

In [25]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_x.shape

torch.Size([12396, 784])

In [24]:
train_y = tensor([1] * len(threes) + [0] * len(sevens)).unsqueeze(1)
train_y.shape

torch.Size([12396, 1])

In [28]:
dset = list(zip(train_x, train_y))

In [30]:
valid_x = torch.cat([valid_3, valid_7]).view(-1, 28*28)
valid_y = tensor([1] * len(valid_3) + [0] * len(valid_7)).unsqueeze(1)
valid_dset = list(zip(valid_x, valid_y))

In [31]:
def init_params(size, std = 1.0):
    return (torch.randn(size) * std).requires_grad_()

In [74]:
weights = init_params((28*28, 1))
weights.shape

torch.Size([784, 1])

In [76]:
bias = init_params(1)
bias.ndim

1

In [68]:
def linear1(xb):
    return (xb @ weights) + bias

In [41]:
preds = linear1(train_x)
preds

tensor([[27.1169],
        [25.8012],
        [29.9281],
        ...,
        [ 0.9456],
        [12.7370],
        [-1.5843]], grad_fn=<AddBackward0>)

In [46]:
list(zip(preds, train_y))[-2:]

[(tensor([12.7370], grad_fn=<UnbindBackward0>), tensor([0])),
 (tensor([-1.5843], grad_fn=<UnbindBackward0>), tensor([0]))]

In [43]:
# this is not a 3 digit classifier but both 3 and 7 digit classifier where corrects will be false only when prediction gives 3 but actual answer is 7 or vice-versa.
corrects = (preds > 0.0).float() == train_y
corrects

tensor([[ True],
        [ True],
        [ True],
        ...,
        [False],
        [False],
        [ True]])

In [48]:
# .item() -> number returns the value of tensor as a standard python number
corrects.float().mean().item()

0.5484833717346191

In [61]:
dl = DataLoader(dset, batch_size=256)
valid_dl = DataLoader(valid_dset, batch_size=256)

In [50]:
# in loss, we checked for distance each prediction has from 1 in case of 3 digit and from 0 in case of 7 digit.
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets == 1, 1 - predictions, predictions).mean()

In [62]:
def calc_grad(xb, yb, model):
    preds = model(xb)
    loss = mnist_loss(preds, yb)
    loss.backward()

In [63]:
def train_epoch(model, lr, params):
    for xb, yb in dl:
        calc_grad(xb, yb, model)
        for p in params:
            p.data -= p.grad * lr
            p.grad.zero_()

In [64]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds > 0.5) == yb
    return correct.float().mean()

In [66]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

In [67]:
# training for 1 epoch
lr = 1.
params = weights, bias
train_epoch(linear1, lr, params)
validate_epoch(linear1)

weights mean is 0.07497538626194, bias is tensor([0.3737], requires_grad=True)
weights mean is 0.07505727559328079, bias is tensor([0.3743], requires_grad=True)
weights mean is 0.0751718282699585, bias is tensor([0.3756], requires_grad=True)
weights mean is 0.07548732310533524, bias is tensor([0.3784], requires_grad=True)
weights mean is 0.07563043385744095, bias is tensor([0.3795], requires_grad=True)
weights mean is 0.07564927637577057, bias is tensor([0.3797], requires_grad=True)
weights mean is 0.07570204138755798, bias is tensor([0.3805], requires_grad=True)
weights mean is 0.07571951299905777, bias is tensor([0.3807], requires_grad=True)
weights mean is 0.07589361071586609, bias is tensor([0.3825], requires_grad=True)
weights mean is 0.07613144814968109, bias is tensor([0.3843], requires_grad=True)
weights mean is 0.07619117945432663, bias is tensor([0.3848], requires_grad=True)
weights mean is 0.07633010298013687, bias is tensor([0.3864], requires_grad=True)
weights mean is 0.07

0.8642

In [70]:
for i in range(20):
    print(f'weights mean is {params[0].float().mean()}, bias is {params[1]}')
    train_epoch(linear1, lr, params)
    print(validate_epoch(linear1), end=' ')
    print()

weights mean is 0.034158021211624146, bias is tensor([-0.6060], requires_grad=True)
0.9755 
weights mean is 0.03465525060892105, bias is tensor([-0.6178], requires_grad=True)
0.9755 
weights mean is 0.03513159230351448, bias is tensor([-0.6294], requires_grad=True)
0.976 
weights mean is 0.03558725863695145, bias is tensor([-0.6410], requires_grad=True)
0.9755 
weights mean is 0.03602146357297897, bias is tensor([-0.6524], requires_grad=True)
0.9755 
weights mean is 0.03643304482102394, bias is tensor([-0.6638], requires_grad=True)
0.9755 
weights mean is 0.03682136535644531, bias is tensor([-0.6752], requires_grad=True)
0.9755 
weights mean is 0.03718718886375427, bias is tensor([-0.6866], requires_grad=True)
0.9755 
weights mean is 0.03753310814499855, bias is tensor([-0.6980], requires_grad=True)
0.9755 
weights mean is 0.03786340355873108, bias is tensor([-0.7094], requires_grad=True)
0.9755 
weights mean is 0.03818289935588837, bias is tensor([-0.7209], requires_grad=True)
0.9755 

In [71]:
# creating optimizer
# nn.Linear does same as init_params that initialized the parameters(weights + bias) and linear(performs xb@w + b)
linear_model = nn.Linear(28*28, 1)

In [77]:
w, b = linear_model.parameters()
w.shape, b.shape

(torch.Size([1, 784]), torch.Size([1]))